In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 15:25:38.928311 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 15:25:38.935813 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 15:25:38.942960 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 15:25:38.949832 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 15:25:38.954389 140593656977216 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 15:25:38.963882 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 15:25:39.003683 140593656977216 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 15:25:39.073235 140593656977216 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9325 train_acc= 0.3286 val_loss= 1.9331 val_acc= 0.3600 time= 1.0606
Epoch: 0002 train_loss= 1.9189 train_acc= 0.2929 val_loss= 1.9200 val_acc= 0.3500 time= 0.0242
Epoch: 0003 train_loss= 1.9036 train_acc= 0.2929 val_loss= 1.9057 val_acc= 0.3500 time= 0.0272
Epoch: 0004 train_loss= 1.8880 train_acc= 0.2929 val_loss= 1.8912 val_acc= 0.3500 time= 0.0234
Epoch: 0005 train_loss= 1.8725 train_acc= 0.2929 val_loss= 1.8769 val_acc= 0.3500 time= 0.0239
Epoch: 0006 train_loss= 1.8565 train_acc= 0.2929 val_loss= 1.8627 val_acc= 0.3500 time= 0.0238
Epoch: 0007 train_loss= 1.8401 train_acc= 0.2929 val_loss= 1.8484 val_acc= 0.3500 time= 0.0191
Epoch: 0008 train_loss= 1.8234 train_acc= 0.2929 val_loss= 1.8340 val_acc= 0.3500 time= 0.0266
Epoch: 0009 train_loss= 1.8071 train_acc= 0.2929 val_loss= 1.8203 val_acc= 0.3500 time= 0.0226
Epoch: 0010 train_loss= 1.7906 train_acc= 0.2929 val_loss= 1.8068 val_acc= 0.3533 time= 0.0202
Epoch: 0011 train_loss= 1.7747 train_acc= 0.3214 v

Epoch: 0097 train_loss= 0.8502 train_acc= 0.8500 val_loss= 1.1273 val_acc= 0.7267 time= 0.0231
Epoch: 0098 train_loss= 0.8437 train_acc= 0.8500 val_loss= 1.1223 val_acc= 0.7367 time= 0.0246
Epoch: 0099 train_loss= 0.8372 train_acc= 0.8500 val_loss= 1.1168 val_acc= 0.7367 time= 0.0191
Epoch: 0100 train_loss= 0.8309 train_acc= 0.8500 val_loss= 1.1112 val_acc= 0.7333 time= 0.0197
Epoch: 0101 train_loss= 0.8243 train_acc= 0.8500 val_loss= 1.1047 val_acc= 0.7333 time= 0.0255
Epoch: 0102 train_loss= 0.8179 train_acc= 0.8500 val_loss= 1.0983 val_acc= 0.7333 time= 0.0233
Epoch: 0103 train_loss= 0.8116 train_acc= 0.8571 val_loss= 1.0930 val_acc= 0.7367 time= 0.0244
Epoch: 0104 train_loss= 0.8053 train_acc= 0.8571 val_loss= 1.0880 val_acc= 0.7433 time= 0.0187
Epoch: 0105 train_loss= 0.7990 train_acc= 0.8571 val_loss= 1.0826 val_acc= 0.7433 time= 0.0280
Epoch: 0106 train_loss= 0.7927 train_acc= 0.8643 val_loss= 1.0781 val_acc= 0.7433 time= 0.0251
Epoch: 0107 train_loss= 0.7869 train_acc= 0.8643 v

Epoch: 0191 train_loss= 0.4794 train_acc= 0.9571 val_loss= 0.8311 val_acc= 0.7867 time= 0.0201
Epoch: 0192 train_loss= 0.4763 train_acc= 0.9571 val_loss= 0.8321 val_acc= 0.7967 time= 0.0234
Epoch: 0193 train_loss= 0.4747 train_acc= 0.9643 val_loss= 0.8333 val_acc= 0.7967 time= 0.0231
Epoch: 0194 train_loss= 0.4735 train_acc= 0.9571 val_loss= 0.8341 val_acc= 0.8067 time= 0.0235
Epoch: 0195 train_loss= 0.4713 train_acc= 0.9571 val_loss= 0.8311 val_acc= 0.8167 time= 0.0245
Epoch: 0196 train_loss= 0.4694 train_acc= 0.9643 val_loss= 0.8274 val_acc= 0.8100 time= 0.0234
Epoch: 0197 train_loss= 0.4683 train_acc= 0.9643 val_loss= 0.8241 val_acc= 0.8167 time= 0.0254
Epoch: 0198 train_loss= 0.4672 train_acc= 0.9643 val_loss= 0.8219 val_acc= 0.8200 time= 0.0190
Epoch: 0199 train_loss= 0.4646 train_acc= 0.9643 val_loss= 0.8205 val_acc= 0.8233 time= 0.0234
Epoch: 0200 train_loss= 0.4614 train_acc= 0.9643 val_loss= 0.8198 val_acc= 0.8233 time= 0.0245


In [13]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8800
accuracy = 0.8030


# Gaussian Graph Convolution

In [14]:
H = Dropout(0.5)(X_in)
H = GaussianGraphConvolution(16, A.shape[0], True, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GaussianGraphConvolution(y.shape[1], A.shape[0], False, support, activation='softmax')([H]+G)

In [15]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [16]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9439 train_acc= 0.3214 val_loss= 1.9447 val_acc= 0.2933 time= 0.3358
Epoch: 0002 train_loss= 1.9411 train_acc= 0.5000 val_loss= 1.9426 val_acc= 0.4400 time= 0.0228
Epoch: 0003 train_loss= 1.9373 train_acc= 0.5071 val_loss= 1.9396 val_acc= 0.4400 time= 0.0178
Epoch: 0004 train_loss= 1.9323 train_acc= 0.5714 val_loss= 1.9358 val_acc= 0.4767 time= 0.0179
Epoch: 0005 train_loss= 1.9262 train_acc= 0.5857 val_loss= 1.9311 val_acc= 0.4933 time= 0.0178
Epoch: 0006 train_loss= 1.9193 train_acc= 0.5857 val_loss= 1.9257 val_acc= 0.5167 time= 0.0225
Epoch: 0007 train_loss= 1.9120 train_acc= 0.6000 val_loss= 1.9203 val_acc= 0.5267 time= 0.0184
Epoch: 0008 train_loss= 1.9043 train_acc= 0.6143 val_loss= 1.9146 val_acc= 0.5267 time= 0.0236
Epoch: 0009 train_loss= 1.8963 train_acc= 0.6143 val_loss= 1.9088 val_acc= 0.5267 time= 0.0270
Epoch: 0010 train_loss= 1.8878 train_acc= 0.6357 val_loss= 1.9025 val_acc= 0.5333 time= 0.0186
Epoch: 0011 train_loss= 1.8786 train_acc= 0.6429 v

Epoch: 0091 train_loss= 0.8194 train_acc= 0.9071 val_loss= 1.0980 val_acc= 0.7700 time= 0.0269
Epoch: 0092 train_loss= 0.8118 train_acc= 0.9071 val_loss= 1.0919 val_acc= 0.7700 time= 0.0259
Epoch: 0093 train_loss= 0.8044 train_acc= 0.9071 val_loss= 1.0861 val_acc= 0.7733 time= 0.0187
Epoch: 0094 train_loss= 0.7971 train_acc= 0.9071 val_loss= 1.0802 val_acc= 0.7700 time= 0.0188
Epoch: 0095 train_loss= 0.7899 train_acc= 0.9071 val_loss= 1.0746 val_acc= 0.7700 time= 0.0177
Epoch: 0096 train_loss= 0.7830 train_acc= 0.9071 val_loss= 1.0691 val_acc= 0.7700 time= 0.0184
Epoch: 0097 train_loss= 0.7761 train_acc= 0.9071 val_loss= 1.0637 val_acc= 0.7733 time= 0.0208
Epoch: 0098 train_loss= 0.7695 train_acc= 0.9000 val_loss= 1.0586 val_acc= 0.7733 time= 0.0199
Epoch: 0099 train_loss= 0.7631 train_acc= 0.9071 val_loss= 1.0538 val_acc= 0.7767 time= 0.0195
Epoch: 0100 train_loss= 0.7567 train_acc= 0.9071 val_loss= 1.0489 val_acc= 0.7767 time= 0.0235
Epoch: 0101 train_loss= 0.7507 train_acc= 0.9071 v

Epoch: 0188 train_loss= 0.4503 train_acc= 0.9714 val_loss= 0.8098 val_acc= 0.8233 time= 0.0191
Epoch: 0189 train_loss= 0.4482 train_acc= 0.9714 val_loss= 0.8089 val_acc= 0.8233 time= 0.0249
Epoch: 0190 train_loss= 0.4461 train_acc= 0.9714 val_loss= 0.8079 val_acc= 0.8200 time= 0.0216
Epoch: 0191 train_loss= 0.4440 train_acc= 0.9714 val_loss= 0.8069 val_acc= 0.8233 time= 0.0202
Epoch: 0192 train_loss= 0.4420 train_acc= 0.9714 val_loss= 0.8062 val_acc= 0.8233 time= 0.0199
Epoch: 0193 train_loss= 0.4400 train_acc= 0.9714 val_loss= 0.8052 val_acc= 0.8233 time= 0.0222
Epoch: 0194 train_loss= 0.4379 train_acc= 0.9714 val_loss= 0.8040 val_acc= 0.8233 time= 0.0187
Epoch: 0195 train_loss= 0.4360 train_acc= 0.9714 val_loss= 0.8029 val_acc= 0.8233 time= 0.0197
Epoch: 0196 train_loss= 0.4339 train_acc= 0.9714 val_loss= 0.8013 val_acc= 0.8233 time= 0.0256
Epoch: 0197 train_loss= 0.4319 train_acc= 0.9714 val_loss= 0.7994 val_acc= 0.8267 time= 0.0226
Epoch: 0198 train_loss= 0.4300 train_acc= 0.9714 v

In [17]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.8440
accuracy = 0.8210


# Attacks